In [ ]:
from datasets import load_dataset
dataset = load_dataset("Graphcore/vqa" , split="validation[:200]")

1. Nghich data xem thử data như thế nào.
2. label2id và id2label.
3. replace string answer to its id.
4. processing data. (tokenize word and image)
5. hugging face trainer to train model.
6. Run inference.
7. run generate blip.

In [ ]:
dataset[15]

In [ ]:
from PIL import Image 
Image.open (dataset[15]['image_id'])

In [ ]:
dataset[0]['label']

## label2id and id2label

In [ ]:
label = list ( set ( [ids  for item in dataset for ids in item['label']['ids']] ) ) 
label2id = { label[i] : i for i in range (len (label)) }
id2label = { v : k for k , v in label2id.items()}

In [ ]:
dataset[0]

In [ ]:
dataset[0]['label']['ids']

## remove cols

In [ ]:
dataset = dataset.remove_columns(["question_type", "question_id" , "answer_type"])
dataset

# convert label

In [ ]:
def convert_ids (sample) :
    sample['label']['ids'] = [label2id [ids] for ids in sample['label']['ids']]
    return sample

In [ ]:
convert_id_data = dataset.map (convert_ids )
flat_data = convert_id_data.flatten ()

In [ ]:
flat_data.features

## Processing data

In [ ]:
# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# text = "a bunch of [MASK] laying on a [MASK]."

# processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
# model = ViltForMaskedLM.from_pretrained("dandelin/vilt-b32-mlm")

# # prepare inputs
# encoding = processor(image, text, return_tensors="pt")

# # forward pass
# outputs = model(**encoding)

# tl = len(re.findall("\[MASK\]", text))
# inferred_token = [text]

# # gradually fill in the MASK tokens, one by one
# with torch.no_grad():
#     for i in range(tl):
#         #tokenize input
#         encoded = processor.tokenizer(inferred_token)
#         # convert to tensor
#         input_ids = torch.tensor(encoded.input_ids).to('cuda')
#         # 
#         encoded = encoded["input_ids"][0][1:-1]
#         outputs = model(input_ids=input_ids, pixel_values=pixel_values)
#         mlm_logits = outputs.logits[0]  # shape (seq_len, vocab_size)
#         # only take into account text features (minus CLS and SEP token)
#         mlm_logits = mlm_logits[1 : input_ids.shape[1] - 1, :]
#         mlm_values, mlm_ids = mlm_logits.softmax(dim=-1).max(dim=-1)
#         # only take into account text
#         mlm_values[torch.tensor(encoded) != 103] = 0
#         select = mlm_values.argmax().item()
#         encoded[select] = mlm_ids[select].item()
#         inferred_token = [processor.decode(encoded)]

# selected_token = ""
# encoded = processor.tokenizer(inferred_token)
# processor.decode(encoded.input_ids[0], skip_special_tokens=True)


# [input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels']

In [ ]:
from transformers import ViltProcessor, ViltForMaskedLM
import requests
from PIL import Image
import re
import torch
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")
# encoding = processor(image, text, return_tensors="pt")

In [ ]:
flat_data.features

In [ ]:
def processing (samples) :
    # extract variable
    image_patchs =  samples['image_id']
    question =  samples['question']
    
    label_id = samples ['label.ids']
    label_weights = samples['label.weights']
    
    image = [Image.open (image_patch) for image_patch in image_patchs]
    #Preprocess
    encoding = processor(image, question, return_tensors="pt" , truncation = True , padding = "max_length")
    for k , v in encoding.items () :
        encoding[k] = v.squeeze () 
        
        
    targets = [] 
    for id_s, scores in zip (label_id , label_weights) :
        target = torch.zeros (len (label2id))
        for id_, score in zip (id_s , scores) :
            target[id_] = score
        targets.append (target)
    encoding['labels'] = targets
    return encoding

In [ ]:
import torch

def preprocess_data(examples):
    image_paths = examples['image_id']
    images = [Image.open(image_path) for image_path in image_paths]
    texts = examples['question']    

    encoding = processor(images, texts, padding="max_length", truncation=True)

    for k, v in encoding.items():
          encoding[k] = v
    targets = []

    for labels, scores in zip(examples['label.ids'], examples['label.weights']):
        target = [0] * len(id2label)

        for label, score in zip(labels, scores):
            target[label] = score
        targets.append(target)

    encoding["labels"] = targets
    
#    print (encoding)
    return encoding

In [ ]:
flat_data.map(preprocess_data, batched=True)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import ViltForQuestionAnswering

model = ViltForQuestionAnswering.from_pretrained(model_checkpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id)

In [ ]:
from transformers import TrainingArguments

repo_id = "HelloWorld2307/ViLT"

training_args = TrainingArguments(
    output_dir=repo_id,
    per_device_train_batch_size=4,
    num_train_epochs=20,
    save_steps=200,
    logging_steps=50,
    learning_rate=5e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    tokenizer=processor,
)

In [ ]:
trainer.train() 

In [ ]:
trainer.push_to_hub()

In [1]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "How many cats are there?"

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# prepare inputs
encoding = processor(image, text, return_tensors="pt")

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()
print("Predicted answer:", model.config.id2label[idx])

2024-03-02 03:51:49.548266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 03:51:49.548363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 03:51:49.676923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Predicted answer: 2


In [3]:
encoding

{'input_ids': tensor([[ 101, 2129, 2116, 8870, 2024, 2045, 1029,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]]), 'pixel_values': tensor([[[[ 0.1059,  0.1294,  0.1373,  ..., -0.2000, -0.1922, -0.1922],
          [ 0.0824,  0.1137,  0.1608,  ..., -0.2235, -0.1373, -0.2314],
          [ 0.0980,  0.1765,  0.1529,  ..., -0.1922, -0.2078, -0.2235],
          ...,
          [ 0.8353,  0.8353,  0.8196,  ...,  0.4980,  0.4667,  0.4510],
          [ 0.8118,  0.8039,  0.7882,  ...,  0.0353,  0.0667, -0.1137],
          [ 0.8667,  0.9059,  0.7098,  ..., -0.3490, -0.4039, -0.4275]],

         [[-0.8039, -0.8118, -0.8275,  ..., -0.9137, -0.8824, -0.9137],
          [-0.8118, -0.8039, -0.7961,  ..., -0.9059, -0.8824, -0.8980],
          [-0.7961, -0.7569, -0.7882,  ..., -0.8745, -0.8588, -0.9059],
          ...,
          [-0.2627, -0.2627, -0.2549,  ..., -0.5686, -0.5216, -0.5608],
          [-0.3176, -0.3176, -0.2863,  ..., -0.764

In [15]:
import torch
import torch.nn as nn

In [35]:
model.vilt.embeddings.text_embeddings.add_module (module = nn.Linear (1520,3000) , name = "dropout")

In [36]:
encoding['input_ids']

tensor([[ 101, 2129, 2116, 8870, 2024, 2045, 1029,  102]])

In [37]:
model.vilt.embeddings.text_embeddings.forward (encoding['input_ids']).shape

torch.Size([1, 8, 1520])

In [47]:
model.vilt.embeddings.text_embeddings.

AttributeError: 'TextEmbeddings' object has no attribute 'TextEmbeddings'

In [29]:
model

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (Linear): Linear(in_features=768, out_features=1520, bias=True)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, o

In [49]:
from transformers import AutoModel
model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [56]:
model.embeddings.add_module (module = nn.Sequential (nn.Linear (768 , 1520) , nn.Dropout (0.1)) , name = "dropout")

In [59]:
model.embeddings.forward(encoding['input_ids']).shape

torch.Size([1, 8, 1520])

In [62]:
model.embeddings.get_submodule

<bound method Module.get_submodule of Embeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Sequential(
    (0): Linear(in_features=768, out_features=1520, bias=True)
    (1): Dropout(p=0.1, inplace=False)
  )
)>

In [57]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Sequential(
      (0): Linear(in_features=768, out_features=1520, bias=True)
      (1): Dropout(p=0.1, inplace=False)
    )
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False